In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv("output/vader_labelled6.csv")
df.head(10)

,full_text,created_at,cleaned,translated,case_fold,token,stop,lemmatized,Positive,Negative,Neutral,Compound,Sentiment
0,Inilah 2 unit Rumah Tapak Jabatan Menteri yang...,Fri Mar 29 11:48:49 +0000 2024,Inilah 2 unit Rumah Tapak Jabatan Menteri yang...,b These are the 2 units of Landed Houses for M...,b these are the units of landed houses for mi...,"['b', 'these', 'are', 'the', 'units', 'of', 'l...","['b', 'units', 'landed', 'houses', 'ministeria...","['b', 'unit', 'land', 'house', 'ministerial', ...",0.000,0.000,1.000,0.0000,Neutral
1,Alhamdulillah terima kasih berkenan mengunjung...,Fri Mar 29 02:34:41 +0000 2024,Alhamdulillah terima kasih berkenan mengunjung...,b Alhamdulillah thank you for visiting the Ind...,b alhamdulillah thank you for visiting the ind...,"['b', 'alhamdulillah', 'thank', 'you', 'for', ...","['b', 'alhamdulillah', 'thank', 'visiting', 'i...","['b', 'alhamdulillah', 'thank', 'visit', 'indo...",0.161,0.000,0.839,0.6369,Positive
2,Pengamat Kritik 2 Proyek Milik Konglomerat Pen...,Thu Mar 28 23:39:00 +0000 2024,Pengamat Kritik 2 Proyek Milik Konglomerat Pen...,b Observers Criticize 2 Projects Owned by Cong...,b observers criticize projects owned by congl...,"['b', 'observers', 'criticize', 'projects', 'o...","['b', 'observers', 'criticize', 'projects', 'o...","['b', 'observer', 'criticize', 'project', 'own...",0.141,0.127,0.732,0.0772,Positive
3,Wow‼️Terlihat dari pinggir jalan Gedung-gedung...,Thu Mar 28 06:04:01 +0000 2024,Wow‼️Terlihat dari pinggir jalan Gedunggedung ...,b Wow Seen from the side of the road towering ...,b wow seen from the side of the road towering ...,"['b', 'wow', 'seen', 'from', 'the', 'side', 'o...","['b', 'wow', 'seen', 'side', 'road', 'towering...","['b', 'wow', 'see', 'side', 'road', 'tower', '...",0.132,0.000,0.868,0.5859,Positive
4,Waktu ngedrone roll off trafo PLN di pelabuhan...,Thu Mar 28 03:03:32 +0000 2024,Waktu ngedrone roll off trafo PLN di pelabuhan...,b When droned to roll off a PLN transformer at...,b when droned to roll off a pln transformer at...,"['b', 'when', 'droned', 'to', 'roll', 'off', '...","['b', 'droned', 'roll', 'pln', 'transformer', ...","['b', 'drone', 'roll', 'pln', 'transformer', '...",0.072,0.000,0.928,0.3400,Positive
5,Usulan DPR tak Ikut Pindah ke IKN Nusantara Di...,Wed Mar 27 03:18:10 +0000 2024,Usulan DPR tak Ikut Pindah ke IKN Nusantara Di...,b DPR s proposal not to participate in moving ...,b dpr s proposal not to participate in moving ...,"['b', 'dpr', 's', 'proposal', 'not', 'to', 'pa...","['b', 'dpr', 'proposal', 'participate', 'movin...","['b', 'dpr', 'proposal', 'participate', 'move'...",0.000,0.107,0.893,-0.3400,Negative
6,Suasana Proyek Pembangunan Bandara VVIP IKN ap...,Wed Mar 27 02:48:24 +0000 2024,Suasana Proyek Pembangunan Bandara VVIP IKN ap...,b The atmosphere of the VVIP IKN Airport Devel...,b the atmosphere of the vvip ikn airport devel...,"['b', 'the', 'atmosphere', 'of', 'the', 'vvip'...","['b', 'atmosphere', 'vvip', 'ikn', 'airport', ...","['b', 'atmosphere', 'vvip', 'ikn', 'airport', ...",0.000,0.000,1.000,0.0000,Neutral
7,Investasi di IKN Nusantara Dinilai Tidak Mengu...,Tue Mar 26 23:39:00 +0000 2024,Investasi di IKN Nusantara Dinilai Tidak Mengu...,b Investment in IKN Nusantara is considered no...,b investment in ikn nusantara is considered no...,"['b', 'investment', 'in', 'ikn', 'nusantara', ...","['b', 'investment', 'ikn', 'nusantara', 'consi...","['b', 'investment', 'ikn', 'nusantara', 'consi...",0.000,0.246,0.754,-0.6002,Negative
8,Inilah Hunian ASN 4 yang mana pembangunan nya ...,Tue Mar 26 13:56:24 +0000 2024,Inilah Hunian ASN 4 yang mana pembangunan nya ...,b This is the ASN 4 Residence where constructi...,b this is the asn residence where constructio...,"['b', 'this', 'is', 'the', 'asn', 'residence',...","['b', 'asn', 'residence', 'construction', 'bec...","['b', 'asn', 'residence', 'construction', 'bec...",0.000,0.000,1.000,0.0000,Neutral
9,Investasi di IKN Nusantara Dinilai Tidak Mengu...,Tue Mar 26 11:47:44 +0000 2

In [6]:
# Assuming 'cleaned' column contains the text data and 'Sentiment' contains labels
texts = df['lemmatized'].astype(str).tolist()
labels = df['Sentiment'].tolist()

# Tokenize the text
tokenizer = Tokenizer(num_words=10000)  # Using the top 10,000 words
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequences
maxlen = 100  # Define the max length of the sequences
X = pad_sequences(sequences, maxlen=maxlen)
y = pd.get_dummies(labels).values  # Convert labels to one-hot encoding


In [7]:
def create_cnn_model(input_length):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=input_length))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=5))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))  # Assuming 3 classes for Sentiment
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [8]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
smote = SMOTE()

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
auc_scores = []

for train_index, test_index in skf.split(X, labels):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Resample the training data
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train.argmax(axis=1))
    y_train_resampled = pd.get_dummies(y_train_resampled).values

    # Create and train the model
    model = create_cnn_model(input_length=maxlen)
    model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = y_pred.argmax(axis=-1)
    y_true = y_test.argmax(axis=-1)

    accuracy_scores.append(accuracy_score(y_true, y_pred_classes))
    precision_scores.append(precision_score(y_true, y_pred_classes, average='weighted'))
    recall_scores.append(recall_score(y_true, y_pred_classes, average='weighted'))
    f1_scores.append(f1_score(y_true, y_pred_classes, average='weighted'))
    auc_scores.append(roc_auc_score(y_true, y_pred, multi_class='ovr'))

print(f'Average Accuracy: {np.mean(accuracy_scores)}')
print(f'Average Precision: {np.mean(precision_scores)}')
print(f'Average Recall: {np.mean(recall_scores)}')
print(f'Average F1 Score: {np.mean(f1_scores)}')
print(f'Average AUC Score: {np.mean(auc_scores)}')


C:\Users\Bryant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


C:\Users\Bryant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


C:\Users\Bryant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


C:\Users\Bryant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


C:\Users\Bryant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Average Accuracy: 0.7665211062590975
Average Precision: 0.7908813505182789
Average Recall: 0.7665211062590975
Average F1 Score: 0.7732951884065642
Average AUC Score: 0.9011603159963721
